# MIMIC 4 data - dataset construction prescriptions

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 300)

In [3]:
adm = pd.read_csv("/path/admissions_processed.csv")
adm.tail()

,Unnamed: 0,subject_id,anchor_age,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag,elapsed_time,elapsed_days,deathtag
16567,524311,19959116,47,25551829,2128-11-06 10:25:00,2128-11-14 17:30:00,NaN,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,?,MARRIED,HISPANIC/LATINO,2128-11-06 07:39:00,2128-11-06 11:51:00,0,8 days 07:05:00,8,0
16568,524320,19962250,58,23717024,2125-12-03 02:27:00,2125-12-11 12:58:00,NaN,OBSERVATION ADMIT,EMERGENCY ROOM,HOME HEALTH CARE,Other,ENGLISH,MARRIED,WHITE,2125-12-03 01:11:00,2125-12-03 04:07:00,0,8 days 10:31:00,8,0
16569,524446,19984270,72,20925564,2185-10-29 18:24:00,2185-11-07 15:56:00,NaN,URGENT,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Medicare,ENGLISH,MARRIED,WHITE,NaN,NaN,0,8 days 21:32:00,8,0
16570,524486,19992202,80,20329411,2153-02-26 00:48:00,2153-03-02 17:00:00,NaN,URGENT,TRANSFER FROM SKILLED NURSING FACILITY,SKILLED NURSING FACILITY,Medicare,ENGLISH,DIVORCED,WHITE,2153-02-25 18:11:00,2153-02-26 01:58:00,0,4 days 16:12:00,4,0
16571,524518,19999068,63,21606769,2161-08-24 04:10:00,2161-09-02 19:00:00,NaN,EW EMER.,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,ENGLISH,DIVORCED,WHITE,2161-08-24 03:14:00,2161-08-24 05:26:00,0,9 days 14:50:00,9,0


In [17]:
presc=pd.read_csv("/path/prescriptions.csv.gz")

#Restrict the dataset to the previously selected admission ids only.
adm_ids=list(adm["hadm_id"])
presc=presc.loc[presc["hadm_id"].isin(adm_ids)]

print("Number of patients remaining in the database: ")
print(presc["subject_id"].nunique())
presc.tail()

Number of patients remaining in the database: 
16572


,subject_id,hadm_id,pharmacy_id,starttime,stoptime,drug_type,drug,gsn,ndc,prod_strength,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route
17021312,16400011,28438471,30081534,2113-06-11 12:00:00,2113-06-12 11:00:00,MAIN,INV-Nimodipine 600mg,NaN,0.000000e+00,600 mg in 0.9% NaCl,NaN,600,mg\ 0 mg,1,DEV,1.0,IVT
17021356,11641713,29862044,74698536,2163-03-30 16:00:00,2163-03-30 15:00:00,MAIN,Ornithine phenylacetate,NaN,0.000000e+00,VIAL,NaN,1,dose,1,dose,1.0,IV
17021357,11641713,29862044,51103598,2163-03-30 16:00:00,2163-04-03 13:00:00,MAIN,Ornithine phenylacetate,NaN,0.000000e+00,VIAL,NaN,1,dose,1,dose,1.0,IV
17021394,19873281,29871982,71573922,2134-03-13 17:00:00,2134-03-16 12:00:00,MAIN,Heparin,006549,6.332303e+10,5000 Units / mL- 1mL Vial,NaN,1800-3500,UNIT,0.36-0.7,mL,NaN,IV
17021397,17961555,22879995,50647416,2171-01-24 05:00:00,2171-02-03 20:00:00,MAIN,Acetaminophen-Caff-Butalbital,004451,5.913369e+08,1 Tablet,NaN,1-2,TAB,1-2,TAB,NaN,PO


In [5]:
#This part is for selecting the x most frequent prescriptions. Instead we use the list of prescriptions as in the paper.

#n_best=10
#For each item, evaluate the number of patients who have been given this item.
#pat_for_item=presc.groupby("drug")["subject_id"].nunique()
#Order by occurence and take the 20 best (the ones with the most patients)
#frequent_labels=pat_for_item.sort_values(ascending=False)[:n_best]
#print(frequent_labels)
#Select only the time series with high occurence.
#presc2=presc.loc[presc["drug"].isin(list(frequent_labels.index))].copy()

#print("Number of patients remaining in the database: ")
#print(presc2["subject_id"].nunique())

In [12]:
#Select entries whose drug name is in the list from the paper.
drugs_list=["Acetaminophen", "Aspirin","Bisacodyl","Insulin","Heparin","Docusate Sodium","D5W","Humulin-R Insulin","Potassium Chloride","Magnesium Sulfate","Metoprolol Tartrate","Sodium Chloride 0.9%  Flush","Pantoprazole"]
presc2=presc.loc[presc["drug"].isin(drugs_list)]

print("Number of patients remaining in the database: ")
print(presc2["subject_id"].nunique())

Number of patients remaining in the database: 
16572


In [13]:
print(presc2.groupby("drug")["dose_unit_rx"].value_counts())

drug                         dose_unit_rx
Acetaminophen                mg              34806
                             g                   6
Aspirin                      mg              12392
Bisacodyl                    mg              17189
D5W                          mL               8595
                             SYR                15
                             ml                  5
                             BTL                 1
Docusate Sodium              mg              22994
Heparin                      UNIT            22015
Insulin                      UNIT            49772
                             units               1
Magnesium Sulfate            gm              40136
Metoprolol Tartrate          mg              45500
Pantoprazole                 mg               8833
Potassium Chloride           mEq             68566
                             mL                  1
Sodium Chloride 0.9%  Flush  mL              47978
                             ml         

In [15]:

#Verification that all input labels have the same amounts units.

#Units correction
presc2=presc2.drop(presc2.loc[presc2["dose_unit_rx"].isnull()].index).copy()
presc2=presc2.drop(presc2.loc[(presc2["drug"]=="Acetaminophen")&(presc2["dose_unit_rx"]!="mg")].index).copy()
presc2.loc[(presc2["drug"]=="D5W")&(presc2["dose_unit_rx"]=="ml"),"dose_unit_rx"]="mL"
presc2=presc2.drop(presc2.loc[(presc2["drug"]=="D5W")&(presc2["dose_unit_rx"]!="mL")].index).copy()
presc2=presc2.drop(presc2.loc[(presc2["drug"]=="Heparin")&(presc2["dose_unit_rx"]!="UNIT")].index).copy()
presc2=presc2.drop(presc2.loc[(presc2["drug"]=="Insulin")&(presc2["dose_unit_rx"]!="UNIT")].index).copy()
presc2=presc2.drop(presc2.loc[(presc2["drug"]=="Magnesium Sulfate")&(presc2["dose_unit_rx"]!="gm")].index).copy()
presc2=presc2.drop(presc2.loc[(presc2["drug"]=="Potassium Chloride")&(presc2["dose_unit_rx"]!="mEq")].index).copy()
presc2.loc[(presc2["drug"]=="Sodium Chloride 0.9%  Flush")&(presc2["dose_unit_rx"]=="ml"),"dose_unit_rx"]="mL"
presc2=presc2.drop(presc2.loc[(presc2["drug"]=="Bisacodyl")&(presc2["dose_unit_rx"]!="mg")].index).copy()
presc2=presc2.drop(presc2.loc[(presc2["drug"]=="Pantoprazole")&(presc2["dose_unit_rx"]!="mg")].index).copy()
print(presc2.groupby("drug")["dose_unit_rx"].value_counts())

drug                         dose_unit_rx
Acetaminophen                mg              34806
Aspirin                      mg              12392
Bisacodyl                    mg              17189
D5W                          mL               8600
Docusate Sodium              mg              22994
Heparin                      UNIT            22015
Insulin                      UNIT            49772
Magnesium Sulfate            gm              40136
Metoprolol Tartrate          mg              45500
Pantoprazole                 mg               8833
Potassium Chloride           mEq             68566
Sodium Chloride 0.9%  Flush  mL              47981
Name: dose_unit_rx, dtype: int64


In [18]:
presc2['charttime']=pd.to_datetime(presc2["starttime"], format='%Y-%m-%d %H:%M:%S')

#To avoid confounding labels with labels from other tables, we add "drug" to the name
presc2["drug"]=presc2["drug"]+" Drug"

In [19]:
presc2.to_csv("/path/prescriptions_processed.csv")

